In [2]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

url_list = []

for i in range(43):
    url = 'https://www.metacritic.com/browse/movie/?releaseYearMin=1910&releaseYearMax=2023&page=' + str(i)
    url_list.append(url)

# len(url_list)

In [3]:
#initializing list with links to each inidividual movie
movie_1000 = []
for i in url_list:
    html = requests.get(i, headers=headers)
    soup = BeautifulSoup(html.content, "html.parser")
    hyperlink_list = soup.find_all('a', class_="c-finderProductCard_container g-color-gray80 u-grid")
    for i in hyperlink_list:
        movie_page = 'https://www.metacritic.com' + i['href']
        movie_1000.append(movie_page)

In [4]:
#len(movie_1000)

In [5]:
#initializing a list called the all_movies that has dictionaries for each movie
all_movies = []  

for i in movie_1000:
    movie_page = requests.get(i, headers=headers)
    movie_page_soup = BeautifulSoup(movie_page.content, "html.parser")
    movie_name_list = movie_page_soup.find_all('div', class_="c-productHero_title g-inner-spacing-bottom-medium g-outer-spacing-top-medium")

    movie_name = ""
    for i in movie_name_list:
        movie_name = i.get_text().strip()

    view_all = movie_page_soup.find_all('a', class_="c-globalHeader_viewMore g-color-gray50 g-text-xxsmall u-text-uppercase")
    for i in view_all:
        view_all_link = 'https://www.metacritic.com' + i['href']

    view_all_page = requests.get(view_all_link, headers=headers)
    view_all_soup = BeautifulSoup(view_all_page.content, "html.parser")

    credit_tables_list = view_all_soup.find_all('div', class_="c-productCredits g-outer-spacing-bottom-xlarge")

    #initializing directors tuple will have all the direcors for a movie
    #which will be used as a key to the sub dictionary which maps to values of the cast members
    directors_tuple = ()
    
    #initializing casts list for each movie
    casts = []  
    for i in credit_tables_list:
        if "Directed By" in i.get_text().strip():
            director_names = i.find_all('a')
            directors_tuple = tuple(j.get_text().strip() for j in director_names)

        table_title = i.find('h3')
        if table_title and table_title.get_text().strip() == 'Cast':
            cast_names = i.find_all('a')
            for j in cast_names:
                cast_name = j.get_text().strip()
                casts.append(cast_name)

    #constructing the movie dictionary and appending it to all_movies list
    movie_dict = {movie_name: {directors_tuple: casts}}
    all_movies.append(movie_dict)

In [6]:
len(all_movies)

1008

In [7]:
#storing my extracted data into a csv file called new_movies.csv
import csv

with open('new_movies.csv', mode='w', newline='') as movie_file:
    movie_writer = csv.writer(movie_file)
    for movie in all_movies:
        for movie_name, details in movie.items():
            for directors_tuple, cast_list in details.items():
                directors_str = ', '.join(directors_tuple)
                cast_str = ', '.join(cast_list)
                movie_writer.writerow([movie_name, directors_str, cast_str])

In [36]:
import numpy as np

#fuunction to calculate cosine similarity
def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        #avoid the division by 0
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

#initializng a dictionary to store directors and their movies
director_to_movies = {}

#processing the data to create a mapping of directors to actors and their frequencies
director_to_actors = {}
for movie in all_movies:
    for movie_name, details in movie.items():
        for directors, cast_list in details.items():
            for director in directors:
                if director not in director_to_actors:
                    director_to_actors[director] = {}
                if director not in director_to_movies:
                    director_to_movies[director] = []
                director_to_movies[director].append(movie_name)
                for actor in cast_list:
                    director_to_actors[director][actor] = director_to_actors[director].get(actor, 0) + 1

while True:
    #this is the main menu
    print('You can choose from 3 options:\n1: Get details about movie\n2: Get details about a director\n3: Get cosine similarity between 2 directors\nType "stop" to exit.')
    choice = input('Enter your choice: ')

    #giving option of exiting to the user
    if choice.lower() == 'stop':
        print("Goodbye!")
        break

    #quesiton 1: get details about a movie
    elif choice == '1':
        while True:
            #user can enter a movie name or go back to the main menu
            choice2 = input('Pick movie name or type "back" to return: ')
            if choice2.lower() == 'back':
                break
            else:
                for i in all_movies:
                    for movie_name, details in i.items():
                        for directors_tuple, cast_list in details.items():
                            if movie_name.lower() == choice2.lower():
                                print('The director(s) of movie',choice2.upper(), 'is/are: ', ', '.join(directors_tuple),"\n")
                                print('The cast members of the movie ',choice2.upper(),' are: ', ', '.join(cast_list))

    #question 2: get details about a director
    elif choice == '2':
        while True:
            #user can enter a director name or go back to the main menu
            choice2 = input('Enter director name or type "back" to return: ')
            if choice2.lower() == 'back':
                break
            else:
                movie_list = []
                cast_list1 = []
                for i in all_movies:
                    for movie_name, details in i.items():
                        for directors_tuple, cast_list in details.items():
                            if choice2 in directors_tuple:
                                movie_list.append(movie_name)
                                cast_list1.extend(cast_list)
                print("Movies directed by", choice2, "are: ",', '.join(movie_list))
                print("Cast of movies directed by:", ', '.join(cast_list1))


    #questiosn 3: get cosine similarity between 2 directors
    elif choice == '3':
        director1 = input("Enter the first director's name: ")
        director2 = input("Enter the second director's name: ")

        if director1 not in director_to_actors or director2 not in director_to_actors:
            print("One or both directors not found in the database.")
        else:
            #creating vectors for each director
            all_actors = set(director_to_actors[director1]).union(set(director_to_actors[director2]))
            vec1 = [director_to_actors[director1].get(actor, 0) for actor in all_actors]
            vec2 = [director_to_actors[director2].get(actor, 0) for actor in all_actors]

            #calculating cosine similarity
            similarity = calculate_cosine_similarity(vec1, vec2)
            #displaying the cosine similaerity
            print(f"{director1} and {director2} have a cosine similarity score of {similarity:.3f}")
    
    else:
        print("Wrong choice. Please enter 1, 2, 3, or 'stop'.")

You can choose from 3 options:
1: Get details about movie
2: Get details about a director
3: Get cosine similarity between 2 directors
Type "stop" to exit.
Enter your choice: 1
Pick movie name or type "back" to return: boyhood
The director(s) of movie BOYHOOD is/are:  Richard Linklater 

The cast members of the movie  BOYHOOD  are:  Ellar Coltrane, Patricia Arquette, Elijah Smith, Lorelei Linklater, Steven Chester Prince, Bonnie Cross, Sydney Orta, Libby Villari, Ethan Hawke, Marco Perella, Jamie Howard, Andrew Villarreal, Shane Graham, Tess Allen, Ryan Power, Sharee Fowler, Mark Finn, Charlie Sexton, Byron Jenkins, Holly Moore, David Blackwell, Barbara Chisholm, Matthew Martinez-Arndt, Cassidy Johnson, Cambell Westmoreland, Jennifer Griffin, Garry Peters, Merrilee McCommas, Tamara Jolaine, Jordan Howard, Andrew Bunten, Tyler Strother, Evie Thompson, Brad Hawkins, Savannah Welch, Mika Odom, Sinjin Venegas, Nick Krause, Derek Chase Hickey, Angela Rawna, Megan Devine, Jenni Tooley, Lando